In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
gender_sub_data = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
gender_sub_data.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [11]:
#This is to fill in missing data
modes = train_data.mode().iloc[0]           
train_data.fillna(modes, inplace=True)
train_data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [98]:
#Fix long tailed distribution in Fare data
train_data['LogFare'] = np.log1p(train_data['Fare'])
#np.log1p(train_data['Fare']) == np.log(train_data['Fare']+1)

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


The code snippet below converts categorical variables (like 'Sex', 'Pclass', and 'Embarked') into a format that can be provided to machine learning algorithms. It creates binary (0 or 1) columns for each category (e.g., 'Sex_male', 'Sex_female').

In [13]:
train_data = pd.get_dummies(train_data, columns=['Sex', 'Pclass', 'Embarked'])        
print(train_data.columns)

Index(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2',
       'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')


In [14]:
test_data = pd.get_dummies(test_data, columns=['Sex', 'Pclass', 'Embarked'])        
print(test_data.columns)

Index(['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')


Here, below, you're defining the features (X) that will be used to train the model and the target variable (Y) that you're trying to predict ('Survived').

In [15]:
new_columns_train=['Sex_male', 'Sex_female', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
new_columns_test=['Sex_male', 'Sex_female', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
x_train = train_data[new_columns_train]  # Replace with actual features
y_train = train_data['Survived']  # Replace with target variable (e.g., 'Survived')
x_test = test_data[new_columns_test]

You create an instance of the LogisticRegression model and then fit it using the training data (x_train, y_train).

In [16]:
titanic_model = LogisticRegression()
titanic_model.fit(x_train, y_train)

LogisticRegression()

In [17]:
y_pred = titanic_model.predict(x_train)

This line imports the accuracy_score function from sklearn.metrics and calculates the accuracy of the model by comparing the predicted classes (y_pred_class) with the true classes (y_test). It then prints the accuracy score.

In [18]:
# Calculate accuracy
accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7777777777777778


In [20]:
predictions = titanic_model.predict(x_test)

In [121]:
x_test

,Sex_male,Sex_female,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,True,False,False,False,True,False,True,False
1,False,True,False,False,True,False,False,True
2,True,False,False,True,False,False,True,False
3,True,False,False,False,True,False,False,True
4,False,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...
413,True,False,False,False,True,False,False,True
414,False,True,True,False,False,True,False,False
415,True,False,False,False,True,False,False,True
416,True,False,False,False,True,False,False,True


In [22]:
arr = np.column_stack([test_data['PassengerId'], predictions])

In [24]:
df = pd.DataFrame(arr, columns=['PassengerId', 'Survived'])

In [25]:
df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [26]:
from IPython.display import HTML

df.to_csv('submission.csv', index=False)
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission.csv')

In [138]:
df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
